In [1]:
%reload_ext autoreload

%autoreload 2
import numpy as np
from models import lstm
import torch

In [15]:
# FILES TO CHANGE (This is where your sequence_nlp_harvey.zip etc is) 

NPY_INPUT_DIR = '/Users/ianmagnusson/IITUDND/data/extracted_features/combined_NLP/harvey/kfold'

In [16]:
def run_experiment(num_hidden, learning_rate, data_dir):
    
    # load data from files
    
    X_seq_tfidf_train = np.load(data_dir + 'X_seq_tfidf_train.npz')
    X_seq_tfidf_test = np.load(data_dir + 'X_seq_tfidf_test.npz')
    
    
    X_seq_glove_train = np.load(data_dir + 'X_seq_glove_train.npz')
    X_seq_glove_test = np.load(data_dir + 'X_seq_glove_test.npz')
    
    y_train = torch.Tensor(np.load(data_dir + 'y_train.npy'))
    y_test = torch.Tensor(np.load(data_dir + 'y_test.npy'))
    
    # build lists with features concatenated
    
    X_train = []
    for i in range(len(y_train)):
        file = X_seq_tfidf_train.files[i]
        X_train.append(torch.Tensor(np.concatenate((X_seq_tfidf_train[file], X_seq_glove_train[file]), axis=1)))
        
    X_test = []
    for i in range(len(y_test)):
        file = X_seq_tfidf_test.files[i]
        X_test.append(torch.Tensor(np.concatenate((X_seq_tfidf_test[file], X_seq_glove_test[file]), axis=1)))
    
    # build and train model
    
    model = lstm.LSTM(input_dim=400, hidden_dim=400)
    
    model.learn(X_train, y_train)
    
    # evaluate
    return model.get_accuracy(X_test, y_test)

def cross_validate(num_hidden, learning_rate, data_dir, folds = 10):
    accuracy_sum = 0.0
    for i in range(folds):
        fold_dir = data_dir + str(i) + '/'
        accuracy_sum += run_experiment(num_hidden,learning_rate,data_dir)
        
    return accuracy_sum / folds


In [ ]:
# cross validate search for hyper-parameters
learning_rates = [0.1, 0.01, 0.001]
hidden_dims = [128,256,512]
results0 = np.zeros((len(learning_rates), len(hidden_dims)))
for i, f in enumerate(learning_rates):
    for j, h in enumerate(hidden_dims):
        results0[i,j] = cross_validate(f, h, NPY_INPUT_DIR) # TODO CHECK THESE PARAMS ARE RIGHT
        
np.save('results0.npy', results0) # to save results
print(results0)

